In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from scipy.integrate import quad
import scipy.stats as stats

In [ ]:
# Definición de variables eV

k = 8.617333262e-5 # eV / K
h = 4.135667e-15 # eV*s
h_bar = h/(2*np.pi) # eV*s
m =  0.5*510999 #eV/c^2  #0.5*9.1e-31 # kg
q = 1.602176e-19

Ec = -4.7  #eV
Ef = -5 #eV

L = 40e-9 # M
W = 3*L # M

C_g = 0.1e-15 # Faradio = Q/V = Q^2/eV
C_q = q*q*m*W*L/(2*np.pi*h_bar*h_bar) # Q^2/eV

C_es = C_g+C_q

eta = C_g/C_es
V_t = Ec

# Parte I

In [ ]:
# Funciones de integrales
def int_N(E,U,T,Vds):
    # print(E, (U+Ec)/(k*T))
    if E < Ef/(k*T):
        return 1/np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))+1/(1+np.exp(E-(Ef-Vds)/(k*T))))
    else:
        return 1/np.sqrt(E-(U+Ec)/(k*T))*(np.exp(Ef/(k*T)-E)/(1+np.exp(Ef/(k*T)-E)) + np.exp((Ef-Vds)/(k*T)-E)/(1+np.exp((Ef-Vds)/(k*T)-E)))

def int_I(E,U,T, Vds):
    if E < Ef/(k*T):
        return np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))-1/(1+np.exp(E-(Ef-Vds)/(k*T))))
    else:
        return np.sqrt(E-(U+Ec)/(k*T))*(np.exp(Ef/(k*T)-E)/(1+np.exp(Ef/(k*T)-E)) - np.exp((Ef-Vds)/(k*T)-E)/(1+np.exp((Ef-Vds)/(k*T)-E)))



# Funciones de cálculo
def get_N(U,T, Vds):
    cte = L*np.sqrt(k*T)*np.sqrt(2*m)/h
    integral = quad(int_N,
                    (U+Ec)/(k*T),
                    np.inf, 
                    args=(U, T, Vds))
    # print(cte, integral) 
    return cte*integral[0]

def get_U(N,N_0, V_gs):
    return q*q/C_es*(N-N_0)-V_gs*eta

def get_I(U, T, Vds):
    cte = q*W*4*np.sqrt(2*m*k*T)*k*T/(h*h)
    integral = quad(int_I,
                    (U+Ec)/(k*T),
                    np.inf, 
                    args=(U, T, Vds))
    return cte*integral[0]

# Loop de autoconsistencia
def solve_I(Vds,T,V_gs, tol = 1e-4,lr = 1e-3):
    
    N_0 = get_N(0,T, 0)
    print(N_0)
    U_ = -1
    
    for _ in tqdm(range(int(1e4)), disable=False):
        N = get_N(U_,T, Vds)
        U = get_U(N,N_0,V_gs)

        if abs(U-U_) < tol:
            print('a')
            break

        else:
            U_ = U_ +lr*(U-U_)
    
    I = get_I(U,T,Vds)
    return U, I

U,I = solve_I(0.3, 
        1, 
        0.5, 
        lr = 0.01)
print(U,I)
    

# Parte II

In [ ]:
def I_lineal(Vds, V_gs): # eq 5.60
    cnt = q*W/(h_bar*h_bar*np.pi*np.pi)*np.sqrt(8*m/9)*np.sqrt(q*eta)**3
    # print(np.sqrt(V_gs-V_t-Vds/eta)**3, np.sqrt(V_gs-V_t)**3, cnt, cnt*(np.sqrt(V_gs-V_t)**3-np.sqrt(V_gs-V_t-Vds/eta)**3))
    return cnt*(np.sqrt(V_gs-V_t)**3-np.sqrt(V_gs-V_t-Vds/eta)**3)


def I_saturado(V_gs): # eq 5.61
    cnt =  q*W/(h_bar*h_bar*np.pi*np.pi)*np.sqrt(8*m/9)*np.sqrt(q*eta)**3
    return cnt*np.sqrt(V_gs-V_t)**3


I_lineal = np.vectorize(I_lineal)
I_saturado = np.vectorize(I_saturado)


In [ ]:
np.sqrt(0.5-V_t)**3-np.sqrt(0.5-V_t-0.05/eta)**3

In [ ]:
q*W/(h_bar*h_bar*np.pi*np.pi)*np.sqrt(8*m/9)*np.sqrt(q*eta)**3

In [ ]:
x = np.linspace(0,0.05, 10)
x_ = np.linspace(0.05,0.5,10)

V_gs = 0.5
V_gs_ = np.ones(10)*V_gs

y = I_lineal(x, V_gs)
y_ = I_saturado(V_gs_)

plt.plot(x,y)
plt.plot(x_,y_)

In [ ]:
eta